In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
fake = Faker()
Faker.seed(12345)
random.seed(12345)

In [3]:
def load_datee():
    """
        Loads some fake dates 
        :returns: tuple containing human readable string, machine readable string, and date object
    """
    dt = fake.date_object()

    try:
        human_readable = format_date(dt, format=random.choice(FORMATS),  locale='en_US') # locale=random.choice(LOCALES))
        human_readable = human_readable.lower()
        human_readable = human_readable.replace(',','')
        machine_readable = dt.isoformat()
        
    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, dt

In [4]:
def load_datasett(m):
    """
        Loads a dataset with m examples and vocabularies
        :m: the number of examples to generate
    """
    
    human_vocab = set()
    machine_vocab = set()
    dataset = []
    Tx = 30
    

    for i in tqdm(range(m)):
        h, m, _ = load_datee()
        if h is not None:
            dataset.append((h, m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))
    
    human = dict(zip(sorted(human_vocab) + ['<unk>', '<pad>'], 
                     list(range(len(human_vocab) + 2))))
    inv_machine = dict(enumerate(sorted(machine_vocab)))
    machine = {v:k for k,v in inv_machine.items()}
 
    return dataset, human, machine, inv_machine


In [5]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_datasett(m)

100%|██████████| 10000/10000 [00:00<00:00, 27176.48it/s]


In [6]:
len(dataset)

10000

In [7]:
len(human_vocab)

37

In [8]:
len(machine_vocab)

11

In [9]:
human_vocab

{' ': 0,
 '.': 1,
 '/': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'y': 34,
 '<unk>': 35,
 '<pad>': 36}

In [10]:
machine_vocab

{'-': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10}

In [11]:
inv_machine_vocab

{0: '-',
 1: '0',
 2: '1',
 3: '2',
 4: '3',
 5: '4',
 6: '5',
 7: '6',
 8: '7',
 9: '8',
 10: '9'}

In [12]:
for i in range(10):
    print(dataset[i])

('9 may 1998', '1998-05-09')
('10.11.19', '2019-11-10')
('9/10/70', '1970-09-10')
('saturday april 28 1990', '1990-04-28')
('thursday january 26 1995', '1995-01-26')
('monday march 7 1983', '1983-03-07')
('sunday may 22 1988', '1988-05-22')
('08 jul 2008', '2008-07-08')
('8 sep 1999', '1999-09-08')
('thursday january 1 1981', '1981-01-01')


In [13]:
dataset[0][0]

'9 may 1998'

In [14]:
X=[]
for i in range(len(dataset[0][0])):
    X.append(human_vocab[dataset[0][0][i]])
a=30-len(dataset[0][0])
for i in range(a):
    X.append(36)

In [15]:
print(X)

[12, 0, 24, 13, 34, 0, 4, 12, 12, 11, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]


In [16]:
np.asarray(X)

array([12,  0, 24, 13, 34,  0,  4, 12, 12, 11, 36, 36, 36, 36, 36, 36, 36,
       36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36])

In [17]:
def preprocess_data_X(dataset,human_vocab):
    X_data=[]
    for i in range(len(dataset)):
        X=[]
        for j in range(len(dataset[i][0])):
            X.append(human_vocab[dataset[i][0][j]])
        a=30-len(dataset[i][0])
        for k in range(a):
            X.append(36)
        X=np.asarray(X)
        X_data.append(X)
    return X_data

In [18]:
X=preprocess_data_X(dataset,human_vocab)
X=np.asarray(X)

In [19]:
X.shape

(10000, 30)

In [20]:
def preprocess_data_Y(dataset,machine_vocab):
    Y_data=[]
    for i in range(len(dataset)):
        Y=[]
        for j in range(len(dataset[i][1])):
            Y.append(machine_vocab[dataset[i][1][j]])
        Y=np.asarray(Y)
        Y_data.append(Y)
    return Y_data

In [21]:
Y=preprocess_data_Y(dataset,machine_vocab)
Y=np.asarray(Y)

In [22]:
Y.shape

(10000, 10)

In [23]:
X_oh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
Y_oh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

In [24]:
X_oh.shape

(10000, 30, 37)

In [25]:
Y_oh.shape

(10000, 10, 11)

In [26]:
Tx=30
Ty=10
n_a=32
n_s=64

In [27]:
# creating layers globally for attention mechanism
repeat_layer = RepeatVector(Tx)
concatenate_layer = Concatenate(axis=-1)
dense1_layer = Dense(10,activation='tanh')
dense2_layer = Dense(1,activation='relu')
softmax_layer = Activation(softmax, name='attention_weights')
dot_layer = Dot(axes=1)

In [28]:
print(repeat_layer)
print(concatenate_layer)
print(dense1_layer)
print(dense2_layer)
print(softmax_layer)
print(dot_layer)

In [29]:
def one_timestep_attention(a,s0):
    
    s_prev = repeat_layer(s0)
    concatenated_output = concatenate_layer([a,s_prev])
    e = dense1_layer(concatenated_output)
    energies = dense2_layer(e)
    alphas = softmax_layer(energies)
    context = dot_layer([alphas,a])
    
    return context

In [44]:
# creating layer for model globally
post_attention_LSTM_layer = LSTM(n_s,return_state=True)
# output_layer = Activation(softmax,name='Y_pred')
output_layer = Dense(len(machine_vocab), activation=softmax)

In [107]:
def model(Tx,Ty,n_s,n_a,human_vocab_size):
    outputs=[]
    X=Input(shape=(Tx,human_vocab_size))
    s0 = Input(shape=(n_s, ))
    c0 = Input(shape=(n_s, ))
    s=s0
    c=c0
    a=Bidirectional(LSTM(n_a,return_sequences=True))(X)
    for i in range(Ty):
        context = one_timestep_attention(a,s)
        s,_,c = post_attention_LSTM_layer(context,initial_state=[s,c])
        output = output_layer(s)
        outputs.append(output)
    model = Model(inputs=[X,s0,c0],outputs=outputs)
    return model

In [108]:
model = model(Tx,Ty,n_s,n_a,len(human_vocab))

In [109]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 30, 37)       0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 30, 64)       17920       input_10[0][0]                   
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 30, 64)       0           input_11[0][0]                   
                                                                 lstm_3[20][0]              

                                                                 attention_weights[34][0]         
                                                                 bidirectional_4[0][0]            
                                                                 attention_weights[35][0]         
                                                                 bidirectional_4[0][0]            
                                                                 attention_weights[36][0]         
                                                                 bidirectional_4[0][0]            
                                                                 attention_weights[37][0]         
                                                                 bidirectional_4[0][0]            
                                                                 attention_weights[38][0]         
                                                                 bidirectional_4[0][0]            
          

In [110]:
# model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
opt =  Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [111]:
s0=np.zeros((m,n_s))
c0=np.zeros((m,n_s))
print(s0.shape)
print(c0.shape)

(10000, 64)
(10000, 64)


In [112]:
outputs = list(Y_oh.swapaxes(0,1))

In [113]:
model.fit([X_oh,s0,c0],outputs,epochs=1,batch_size=100)

Epoch 1/1


 2500/10000 [======>.......................] - ETA: 6:46 - loss: 22.2877 - dense_3_loss: 2.6321 - dense_3_accuracy: 0.6400 - dense_3_accuracy_1: 0.0000e+00 - dense_3_accuracy_2: 0.1700 - dense_3_accuracy_3: 0.0800 - dense_3_accuracy_4: 0.4600 - dense_3_accuracy_5: 0.1000 - dense_3_accuracy_6: 0.1100 - dense_3_accuracy_7: 0.0700 - dense_3_accuracy_8: 0.3600 - dense_3_accuracy_9: 0.110 - ETA: 3:25 - loss: 21.3278 - dense_3_loss: 2.8147 - dense_3_accuracy: 0.5950 - dense_3_accuracy_1: 0.2900 - dense_3_accuracy_2: 0.1950 - dense_3_accuracy_3: 0.0500 - dense_3_accuracy_4: 0.7300 - dense_3_accuracy_5: 0.0500 - dense_3_accuracy_6: 0.0800 - dense_3_accuracy_7: 0.4250 - dense_3_accuracy_8: 0.2550 - dense_3_accuracy_9: 0.0950    - ETA: 2:18 - loss: 20.7704 - dense_3_loss: 2.8404 - dense_3_accuracy: 0.5667 - dense_3_accuracy_1: 0.3700 - dense_3_accuracy_2: 0.1900 - dense_3_accuracy_3: 0.0433 - dense_3_accuracy_4: 0.8200 - dense_3_accuracy_5: 0.2567 - dense_3_accuracy_6: 0.0800 - dense_3_accuracy_

 5000/10000 [==============>...............] - ETA: 17s - loss: 13.4344 - dense_3_loss: 2.5381 - dense_3_accuracy: 0.6735 - dense_3_accuracy_1: 0.7623 - dense_3_accuracy_2: 0.3531 - dense_3_accuracy_3: 0.1008 - dense_3_accuracy_4: 0.9792 - dense_3_accuracy_5: 0.7423 - dense_3_accuracy_6: 0.2096 - dense_3_accuracy_7: 0.9096 - dense_3_accuracy_8: 0.3500 - dense_3_accuracy_9: 0.12 - ETA: 16s - loss: 13.2867 - dense_3_loss: 2.5264 - dense_3_accuracy: 0.6811 - dense_3_accuracy_1: 0.7689 - dense_3_accuracy_2: 0.3633 - dense_3_accuracy_3: 0.1044 - dense_3_accuracy_4: 0.9800 - dense_3_accuracy_5: 0.7478 - dense_3_accuracy_6: 0.2130 - dense_3_accuracy_7: 0.9130 - dense_3_accuracy_8: 0.3537 - dense_3_accuracy_9: 0.12 - ETA: 16s - loss: 13.1558 - dense_3_loss: 2.5173 - dense_3_accuracy: 0.6900 - dense_3_accuracy_1: 0.7757 - dense_3_accuracy_2: 0.3693 - dense_3_accuracy_3: 0.1075 - dense_3_accuracy_4: 0.9807 - dense_3_accuracy_5: 0.7500 - dense_3_accuracy_6: 0.2218 - dense_3_accuracy_7: 0.9161 - d

 7500/10000 [=====================>........] - ETA: 7s - loss: 10.9636 - dense_3_loss: 2.3645 - dense_3_accuracy: 0.7978 - dense_3_accuracy_1: 0.8582 - dense_3_accuracy_2: 0.4998 - dense_3_accuracy_3: 0.2235 - dense_3_accuracy_4: 0.9894 - dense_3_accuracy_5: 0.8255 - dense_3_accuracy_6: 0.3275 - dense_3_accuracy_7: 0.9539 - dense_3_accuracy_8: 0.4098 - dense_3_accuracy_9: 0.156 - ETA: 7s - loss: 10.8845 - dense_3_loss: 2.3591 - dense_3_accuracy: 0.8010 - dense_3_accuracy_1: 0.8602 - dense_3_accuracy_2: 0.5037 - dense_3_accuracy_3: 0.2315 - dense_3_accuracy_4: 0.9896 - dense_3_accuracy_5: 0.8273 - dense_3_accuracy_6: 0.3335 - dense_3_accuracy_7: 0.9548 - dense_3_accuracy_8: 0.4110 - dense_3_accuracy_9: 0.157 - ETA: 7s - loss: 10.8057 - dense_3_loss: 2.3538 - dense_3_accuracy: 0.8032 - dense_3_accuracy_1: 0.8617 - dense_3_accuracy_2: 0.5092 - dense_3_accuracy_3: 0.2387 - dense_3_accuracy_4: 0.9898 - dense_3_accuracy_5: 0.8291 - dense_3_accuracy_6: 0.3385 - dense_3_accuracy_7: 0.9557 - de

10000/10000 [==============================] - ETA: 3s - loss: 9.1782 - dense_3_loss: 2.2368 - dense_3_accuracy: 0.8533 - dense_3_accuracy_1: 0.8971 - dense_3_accuracy_2: 0.6134 - dense_3_accuracy_3: 0.4170 - dense_3_accuracy_4: 0.9929 - dense_3_accuracy_5: 0.8695 - dense_3_accuracy_6: 0.4508 - dense_3_accuracy_7: 0.9689 - dense_3_accuracy_8: 0.4630 - dense_3_accuracy_9: 0.19 - ETA: 2s - loss: 9.1203 - dense_3_loss: 2.2318 - dense_3_accuracy: 0.8543 - dense_3_accuracy_1: 0.8979 - dense_3_accuracy_2: 0.6171 - dense_3_accuracy_3: 0.4234 - dense_3_accuracy_4: 0.9930 - dense_3_accuracy_5: 0.8705 - dense_3_accuracy_6: 0.4547 - dense_3_accuracy_7: 0.9694 - dense_3_accuracy_8: 0.4639 - dense_3_accuracy_9: 0.19 - ETA: 2s - loss: 9.0598 - dense_3_loss: 2.2257 - dense_3_accuracy: 0.8556 - dense_3_accuracy_1: 0.8990 - dense_3_accuracy_2: 0.6203 - dense_3_accuracy_3: 0.4296 - dense_3_accuracy_4: 0.9931 - dense_3_accuracy_5: 0.8714 - dense_3_accuracy_6: 0.4577 - dense_3_accuracy_7: 0.9697 - dense_3

In [114]:
model.load_weights('models/model.h5')

In [115]:
prediction = model.predict([X_oh,s0,c0])

In [116]:
print(prediction)

[array([[2.6088838e-11, 6.3822824e-08, 9.9998188e-01, ..., 4.8161457e-09,
        2.9087920e-07, 1.3107003e-05],
       [2.8338750e-08, 2.0119222e-05, 4.0740613e-02, ..., 1.3152727e-08,
        1.5659752e-07, 7.5371742e-07],
       [5.5637277e-09, 3.0860741e-05, 9.7720271e-01, ..., 2.3642508e-05,
        1.7438088e-06, 1.4035256e-05],
       ...,
       [7.6414244e-09, 1.3443561e-05, 1.4835117e-04, ..., 9.6848429e-10,
        7.2417827e-09, 1.0684141e-08],
       [4.0109648e-11, 4.8570438e-08, 9.9997640e-01, ..., 4.4348781e-06,
        2.1020394e-07, 7.8001503e-06],
       [4.4748254e-09, 1.3930190e-05, 4.5602821e-05, ..., 6.9536482e-10,
        6.2133552e-09, 8.1373841e-09]], dtype=float32), array([[1.05072845e-14, 6.98603856e-07, 2.82881530e-07, ...,
        3.45538070e-10, 1.21142392e-07, 9.99998927e-01],
       [5.20977983e-11, 9.95485961e-01, 4.82890231e-04, ...,
        1.93118832e-09, 2.65753126e-08, 3.90636548e-03],
       [2.97418340e-11, 2.23968681e-02, 1.32804344e-05, ...,
 

In [117]:
prediction = np.argmax(prediction, axis = -1)

In [118]:
print(prediction)

[[ 2  3  2 ...  3  2  3]
 [10  1 10 ...  1 10  1]
 [10  2  8 ...  2  8  2]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 1  2  2 ...  2  1  3]
 [10  1  1 ...  8  5  8]]


In [119]:
Y

array([[ 2, 10, 10, ...,  0,  1, 10],
       [ 3,  1,  2, ...,  0,  2,  1],
       [ 2, 10,  8, ...,  0,  2,  1],
       ...,
       [ 3,  1,  2, ...,  0,  2,  8],
       [ 2, 10,  8, ...,  0,  1,  5],
       [ 3,  1,  2, ...,  0,  3,  8]])

In [120]:
prediction.shape

(10, 10000)

In [121]:
Y.shape

(10000, 10)

In [122]:
prediction[0]

array([2, 3, 2, ..., 3, 2, 3], dtype=int64)

In [123]:
Y[0]

array([ 2, 10, 10,  9,  0,  1,  6,  0,  1, 10])

In [126]:
prediction=prediction.swapaxes(0,1)

In [127]:
prediction.shape

(10000, 10)

In [128]:
prediction[0]

array([ 2, 10, 10,  9,  0,  1,  6,  0,  1, 10], dtype=int64)

In [124]:
output=[]
for i in range(len(prediction)):
    out=prediction[i][0]
    output.append(out)

In [125]:
output

[2, 10, 10, 9, 0, 1, 6, 0, 1, 10]

In [130]:
c=0
for i in range(prediction.shape[0]):
    if prediction[i].all()==Y[i].all():
        c=c+1
print(c)

10000
